<a href="https://colab.research.google.com/github/PranavPutsa1006/Deep-Learning/blob/main/CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Apr  3 07:09:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import keras
import numpy as np
from sklearn.model_selection import train_test_split

from keras.utils import np_utils
from keras.datasets import mnist,cifar100
from keras.applications import ResNet50
from keras import Model

import matplotlib.pyplot as plt

In [ ]:
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

169009152/169001437 [==============================] - 3s 0us/step


In [ ]:
# For CIFAR 100 Uncomment the following lines

X_train = X_train.reshape(-1,32,32,3)
X_test = X_test.reshape(-1,32,32,3)

# X_train = X_train.reshape(-1, 28, 28, 1)
# X_test = X_test.reshape(-1, 28, 28, 1)

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
y_trainOneHot = np_utils.to_categorical(y_train)
y_testOneHot = np_utils.to_categorical(y_test)
y_trainOneHot.shape, y_testOneHot.shape

((50000, 100), (10000, 100))

In [ ]:
X_train, X_validate, y_label, y_validate = train_test_split(X_train, y_trainOneHot, test_size = 0.20, random_state = 101)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = (32,32,3)))
# model.add(Conv2D(32, kernel_size = (7,7), activation = 'relu', input_shape = (28, 28, 1)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.20))

model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.20))

model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.20))

model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.20))
# model.add(Dense(10, activation = 'softmax'))
model.add(Dense(100,activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 128)        

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])

In [ ]:
my_callbacks = keras.callbacks.EarlyStopping(monitor="accuracy",patience=2)
history = model.fit(X_train, y_label, epochs = 2000, batch_size=32, validation_data=(X_validate, y_validate), callbacks=my_callbacks)

Epoch 1/2000
1250/1250 [==============================] - 9s 5ms/step - loss: 4.5525 - accuracy: 0.0173 - val_loss: 4.2750 - val_accuracy: 0.0434
Epoch 2/2000
1250/1250 [==============================] - 5s 4ms/step - loss: 4.2397 - accuracy: 0.0483 - val_loss: 3.9992 - val_accuracy: 0.1023
Epoch 3/2000
1250/1250 [==============================] - 5s 4ms/step - loss: 3.9772 - accuracy: 0.0944 - val_loss: 3.8135 - val_accuracy: 0.1306
Epoch 4/2000
1250/1250 [==============================] - 5s 4ms/step - loss: 3.8119 - accuracy: 0.1149 - val_loss: 3.7198 - val_accuracy: 0.1490
Epoch 5/2000
1250/1250 [==============================] - 5s 4ms/step - loss: 3.7199 - accuracy: 0.1290 - val_loss: 3.6097 - val_accuracy: 0.1638
Epoch 6/2000
1250/1250 [==============================] - 5s 4ms/step - loss: 3.6339 - accuracy: 0.1420 - val_loss: 3.5210 - val_accuracy: 0.1817
Epoch 7/2000
1250/1250 [==============================] - 5s 4ms/step - loss: 3.5666 - accuracy: 0.1554 - val_loss: 3.4413 -

In [ ]:
print(history.history.keys())

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])


In [ ]:
valAccuracy = history.history['val_acc']
valLoss = history.history['val_loss']
modelLoss = history.history['loss']
accuracy = history.history['acc']
numEpochs = range(len(accuracy))

In [ ]:
#include plot for tranining accuracy vs test accuracy

In [ ]:
dr_model = Sequential()
dr_model.add(ZeroPadding2D(4, input_shape=(32,32,3)))
# Stack 1:
dr_model.add(Conv2D(384, (3, 3), padding='same'))
dr_model.add(Activation('relu'))
dr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
dr_model.add(Dropout(0.5))
# Stack 2:
dr_model.add(Conv2D(384, (1, 1), padding='same'))
dr_model.add(Conv2D(384, (2, 2), padding='same'))
dr_model.add(Conv2D(640, (2, 2), padding='same'))
dr_model.add(Conv2D(640, (2, 2), padding='same'))
dr_model.add(Activation('relu'))
dr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
dr_model.add(Dropout(0.5))
# Stack 3:
dr_model.add(Conv2D(640, (3, 3), padding='same'))
dr_model.add(Conv2D(768, (2, 2), padding='same'))
dr_model.add(Conv2D(768, (2, 2), padding='same'))
dr_model.add(Conv2D(768, (2, 2), padding='same'))
dr_model.add(Activation('relu'))
dr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
dr_model.add(Dropout(0.5))
# Stack 4:
dr_model.add(Conv2D(768, (1, 1), padding='same'))
dr_model.add(Conv2D(896, (2, 2), padding='same'))
dr_model.add(Conv2D(896, (2, 2), padding='same'))
dr_model.add(Activation('relu'))
dr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
dr_model.add(Dropout(0.5))
# Stack 5:
dr_model.add(Conv2D(896, (3, 3), padding='same'))
dr_model.add(Conv2D(1024, (2, 2), padding='same'))
dr_model.add(Conv2D(1024, (2, 2), padding='same'))
dr_model.add(Activation('relu'))
dr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
dr_model.add(Dropout(0.5))
# Stack 6:
dr_model.add(Conv2D(1024, (1, 1), padding='same'))
dr_model.add(Conv2D(1152, (2, 2), padding='same'))
dr_model.add(Activation('relu'))
dr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
dr_model.add(Dropout(0.5))
# Stack 7:
dr_model.add(Conv2D(1152, (1, 1), padding='same'))
dr_model.add(Activation('relu'))
dr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
dr_model.add(Dropout(0.5))
dr_model.add(Flatten())
dr_model.add(Dense(100))
dr_model.add(Activation('softmax'))

In [ ]:
dr_model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])

In [ ]:
history = dr_model.fit(X_train, y_label, epochs = 50, batch_size=100, validation_data=(X_validate, y_validate))

Epoch 1/50
400/400 [==============================] - 532s 1s/step - loss: 2.5675 - accuracy: 0.3345 - val_loss: 2.3796 - val_accuracy: 0.3803
Epoch 2/50
400/400 [==============================] - 532s 1s/step - loss: 2.5226 - accuracy: 0.3442 - val_loss: 2.3410 - val_accuracy: 0.3894
Epoch 3/50
400/400 [==============================] - 532s 1s/step - loss: 2.4900 - accuracy: 0.3519 - val_loss: 2.4047 - val_accuracy: 0.3765
Epoch 4/50
400/400 [==============================] - 532s 1s/step - loss: 2.4515 - accuracy: 0.3617 - val_loss: 2.3716 - val_accuracy: 0.3854
Epoch 5/50
400/400 [==============================] - 532s 1s/step - loss: 2.4187 - accuracy: 0.3664 - val_loss: 2.2605 - val_accuracy: 0.4032
Epoch 6/50
400/400 [==============================] - 532s 1s/step - loss: 2.3770 - accuracy: 0.3742 - val_loss: 2.2079 - val_accuracy: 0.4184
Epoch 7/50
400/400 [==============================] - 532s 1s/step - loss: 2.3548 - accuracy: 0.3816 - val_loss: 2.2632 - val_accuracy: 0.4095

KeyboardInterrupt: ignored